In [1]:
import pandas as pd
import os
import plotly.express as px
import plotly.graph_objs as go

In [2]:

def plot_sensor_status(df, start_time, end_time, withings_date, granular_date):
    # Forza start_time e end_time ad avere lo stesso fuso orario di local_time
    start_time = pd.to_datetime(start_time).tz_localize(df['local_time'].dt.tz)
    end_time = pd.to_datetime(end_time).tz_localize(df['local_time'].dt.tz)
    
    # Filtrare solo i sensori richiesti
    selected_sensors = ['HouseEntrance', 'MotionBathroom', 'MotionBedroom', 'MotionKitchen', 'MotionLivingroom']
    df_filtered = df[df['sensor_id'].isin(selected_sensors)]
    
    # Filtrare in base a start_time, end_time e solo per i punti con stato "OFF"
    df_filtered = df_filtered[
        (df_filtered['local_time'] >= start_time) &
        (df_filtered['local_time'] <= end_time) &
        (df_filtered['sensor_status'] == 'off')
    ]

    # Creare il grafico con local_time come asse x
    fig = px.scatter(df_filtered, x='local_time', y='sensor_id', color='sensor_status',
                     title='Sensor Status Over Time', labels={'local_time': 'Time', 'sensor_id': 'Sensor'})
    
    fig.update_xaxes(title_text='Date and Time')
    fig.update_yaxes(title_text='Sensor')

    # Calcolare le date specificate
    withings_date = pd.to_datetime(withings_date).tz_localize(df['local_time'].dt.tz)
    granular_date = pd.to_datetime(granular_date).tz_localize(df['local_time'].dt.tz)

    # Aggiungere i punti rossi per ogni sensore nella data specificata
    for sensor in selected_sensors:
        fig.add_scatter(x=[withings_date], y=[sensor], mode='markers', marker=dict(color='red', size=10), name='AggregatedWithings')

    # Aggiungere i punti rossi per ogni sensore nella data specificata
    for sensor in selected_sensors:
        fig.add_scatter(x=[granular_date], y=[sensor], mode='markers', marker=dict(color='green', size=10), name='AggregatedGranular')

    startDate = start_time.date()
    endDate = end_time.date()
    # Salva il grafico come immagine
    fig.write_image(f"/Users/boe/Desktop/Unimi/Tesi/code/graphs/SensorsData/SensorProblem_{startDate}_{endDate}.png",width=1600, height=800)

    # Mostrare il grafico
    fig.show()

# StartValue Mismatch

# January 2024

## Caricamento del dataframe per gennaio

In [ ]:
# Lista delle cartelle (sostituisci con i tuoi percorsi)
folders = [
    '/Users/boe/Desktop/Unimi/Tesi/code/SensorsData/subject_1/environmentals/Dishes_Glasses/',
    '/Users/boe/Desktop/Unimi/Tesi/code/SensorsData/subject_1/environmentals/FoodStorage',
    '/Users/boe/Desktop/Unimi/Tesi/code/SensorsData/subject_1/environmentals/Freezer',
    '/Users/boe/Desktop/Unimi/Tesi/code/SensorsData/subject_1/environmentals/HouseEntrance',
    '/Users/boe/Desktop/Unimi/Tesi/code/SensorsData/subject_1/environmentals/Microwave',
    '/Users/boe/Desktop/Unimi/Tesi/code/SensorsData/subject_1/environmentals/MotionBathroom',
    '/Users/boe/Desktop/Unimi/Tesi/code/SensorsData/subject_1/environmentals/MotionBedroom',
    '/Users/boe/Desktop/Unimi/Tesi/code/SensorsData/subject_1/environmentals/MotionKitchen',
    '/Users/boe/Desktop/Unimi/Tesi/code/SensorsData/subject_1/environmentals/MotionLivingroom',
    '/Users/boe/Desktop/Unimi/Tesi/code/SensorsData/subject_1/environmentals/PlugTvHall',
    '/Users/boe/Desktop/Unimi/Tesi/code/SensorsData/subject_1/environmentals/Refrigerator',
    '/Users/boe/Desktop/Unimi/Tesi/code/SensorsData/subject_1/environmentals/Shower_Hum_Temp_humidity',
    '/Users/boe/Desktop/Unimi/Tesi/code/SensorsData/subject_1/environmentals/Shower_Hum_Temp_temp',
    '/Users/boe/Desktop/Unimi/Tesi/code/SensorsData/subject_1/environmentals/Silverware',
    '/Users/boe/Desktop/Unimi/Tesi/code/SensorsData/subject_1/environmentals/Stove_Hum_Temp_humidity',
    '/Users/boe/Desktop/Unimi/Tesi/code/SensorsData/subject_1/environmentals/Stove_Hum_Temp_humidity',
    '/Users/boe/Desktop/Unimi/Tesi/code/SensorsData/subject_1/environmentals/WashingMachine'
]

# Nome del file che cerchiamo in ogni cartella
filename = "January_2024.csv"

# Lista per contenere tutti i DataFrame
dfs = []

# Legge il CSV da ogni cartella
for folder in folders:
    file_path = os.path.join(folder, filename)
    # Verifica se il file esiste prima di leggerlo
    if os.path.exists(file_path):
        try:
            df = pd.read_csv(file_path)
            dfs.append(df)
        except Exception as e:
            print(f"Errore nella lettura del file {file_path}: {str(e)}")
    else:
        print(f"File non trovato in: {folder}")

# Se abbiamo trovato almeno un file
if dfs:
    # Unisce tutti i DataFrame in uno solo
    df_finale_january = pd.concat(dfs, ignore_index=True)
    
    # Ordina il DataFrame per local_time
    df_finale_january = df_finale_january.sort_values('local_time')
    
    # Resetta l'indice dopo l'ordinamento
    df_finale_january = df_finale_january.reset_index(drop=True)
    
    print(f"Elaborati {len(dfs)} file CSV con successo")
else:
    print("Nessun file CSV trovato nelle cartelle specificate")

## Nottata del 2024-01-07 - 2024-01-08 // Withings: 22:07 - Granular : 00:07

In [ ]:
df_finale_january['local_time'] = pd.to_datetime(df_finale_january['local_time'])
start_time = '2024-01-07 20:00:00'
end_time = '2024-01-08 08:00:00'
WithigsTime = '2024-01-07 22:07:00'  # Primo orario per la linea verticale
Granulartime = '2024-01-08 00:07:00'  # Secondo orario per la linea verticale
plot_sensor_status(df_finale_january, start_time, end_time, WithigsTime, Granulartime)

### per i dati aggregati di Withings dovrebbe essere andato a letto alle 22:07:00 mentre per i dati aggregati dai granulari alle 00:07:00. dal grafico si evince che gli aggregati di Withings sono più precisi perchè non ho nessun dato che mi giustifichi che alle 00:07 si sia addormentato dato che non ho nessun movimento dalle 22:06 alle 01:16

## Nottata del 2024-01-10 - 2024-01-11 // Withings: 21:00 - Granular : 00:04

In [ ]:
df_finale_january['local_time'] = pd.to_datetime(df_finale_january['local_time'])
start_time = '2024-01-10 20:00:00'
end_time = '2024-01-11 08:00:00'
WithigsTime = '2024-01-10 21:00:00'  # Primo orario per la linea verticale
Granulartime = '2024-01-11 00:04:00'  # Secondo orario per la linea verticale
plot_sensor_status(df_finale_january, start_time, end_time, WithigsTime, Granulartime)

### per i dati aggregati di Withings dovrebbe essere andato a letto alle 21:00:00 mentre per i dati aggregati dai granulari alle 00:04:00. dal grafico si evince che gli aggregati di Withings sono più precisi perchè non ho nessun dato che mi giustifichi che alle 00:04 si sia addormentato dato che non ho nessun movimento dalle 23:47 alle 00:28

## Notata del 2024-01-23 - 2024-01-24 // Withings: 22:14 - Granular: 23:02

In [ ]:
df_finale_january['local_time'] = pd.to_datetime(df_finale_january['local_time'])
start_time = '2024-01-23 20:00:00'
end_time = '2024-01-24 08:00:00'
WithigsTime = '2024-01-23 22:14:00'  # Primo orario per la linea verticale
Granulartime = '2024-01-23 23:02:00'  # Secondo orario per la linea verticale
plot_sensor_status(df_finale_january, start_time, end_time, WithigsTime, Granulartime)

### per i dati aggregati di Withings dovrebbe essere andato a letto alle 22:14:00 mentre per i dati aggregati dai granulari alle 23:02:00. dal grafico si evince che gli aggregati di Withings sono più precisi perchè non ho nessun dato che mi giustifichi che alle 23:02 si sia addormentato dato che non ho nessun movimento dalle 22:19 alle 03:14

# May 2024

## Caricamento del dataframe per maggio

In [ ]:
# Lista delle cartelle (sostituisci con i tuoi percorsi)
folders = [
    '/Users/boe/Desktop/Unimi/Tesi/code/SensorsData/subject_1/environmentals/Dishes_Glasses/',
    '/Users/boe/Desktop/Unimi/Tesi/code/SensorsData/subject_1/environmentals/FoodStorage',
    '/Users/boe/Desktop/Unimi/Tesi/code/SensorsData/subject_1/environmentals/Freezer',
    '/Users/boe/Desktop/Unimi/Tesi/code/SensorsData/subject_1/environmentals/HouseEntrance',
    '/Users/boe/Desktop/Unimi/Tesi/code/SensorsData/subject_1/environmentals/Microwave',
    '/Users/boe/Desktop/Unimi/Tesi/code/SensorsData/subject_1/environmentals/MotionBathroom',
    '/Users/boe/Desktop/Unimi/Tesi/code/SensorsData/subject_1/environmentals/MotionBedroom',
    '/Users/boe/Desktop/Unimi/Tesi/code/SensorsData/subject_1/environmentals/MotionKitchen',
    '/Users/boe/Desktop/Unimi/Tesi/code/SensorsData/subject_1/environmentals/MotionLivingroom',
    '/Users/boe/Desktop/Unimi/Tesi/code/SensorsData/subject_1/environmentals/PlugTvHall',
    '/Users/boe/Desktop/Unimi/Tesi/code/SensorsData/subject_1/environmentals/Refrigerator',
    '/Users/boe/Desktop/Unimi/Tesi/code/SensorsData/subject_1/environmentals/Shower_Hum_Temp_humidity',
    '/Users/boe/Desktop/Unimi/Tesi/code/SensorsData/subject_1/environmentals/Shower_Hum_Temp_temp',
    '/Users/boe/Desktop/Unimi/Tesi/code/SensorsData/subject_1/environmentals/Silverware',
    '/Users/boe/Desktop/Unimi/Tesi/code/SensorsData/subject_1/environmentals/Stove_Hum_Temp_humidity',
    '/Users/boe/Desktop/Unimi/Tesi/code/SensorsData/subject_1/environmentals/Stove_Hum_Temp_humidity',
    '/Users/boe/Desktop/Unimi/Tesi/code/SensorsData/subject_1/environmentals/WashingMachine'
]

# Nome del file che cerchiamo in ogni cartella
filename = "May_2024.csv"

# Lista per contenere tutti i DataFrame
dfs = []

# Legge il CSV da ogni cartella
for folder in folders:
    file_path = os.path.join(folder, filename)
    # Verifica se il file esiste prima di leggerlo
    if os.path.exists(file_path):
        try:
            df = pd.read_csv(file_path)
            dfs.append(df)
        except Exception as e:
            print(f"Errore nella lettura del file {file_path}: {str(e)}")
    else:
        print(f"File non trovato in: {folder}")

# Se abbiamo trovato almeno un file
if dfs:
    # Unisce tutti i DataFrame in uno solo
    df_finale_may = pd.concat(dfs, ignore_index=True)
    
    # Ordina il DataFrame per local_time
    df_finale_may = df_finale_may.sort_values('local_time')
    
    # Resetta l'indice dopo l'ordinamento
    df_finale_may = df_finale_may.reset_index(drop=True)
    
    print(f"Elaborati {len(dfs)} file CSV con successo")
else:
    print("Nessun file CSV trovato nelle cartelle specificate")

## Nottata del 2024-05-01 - 2024-05-01 // Withings: 20:15 - Granular : 22:10

In [ ]:
df_finale_may['local_time'] = pd.to_datetime(df_finale_may['local_time'])
start_time = '2024-05-01 20:00:00'
end_time = '2024-05-02 08:00:00'
WithigsTime = '2024-05-01 20:15:00'  # Primo orario per la linea verticale
Granulartime = '2024-05-01 22:10:00'  # Secondo orario per la linea verticale
plot_sensor_status(df_finale_may, start_time, end_time, WithigsTime, Granulartime)

### per i dati aggregati di Withings dovrebbe essere andato a letto alle 20:15:00 mentre per i dati aggregati dai granulari alle 22:10:00. dal grafico si evince che gli aggregati di Withings sono più precisi perchè non ho nessun dato che mi giustifichi che alle 22:10 si sia addormentato dato che non ho nessun movimento dalle 21:56 alle 23:52

# EndValue Mismatch

# February

## Caricamento del dataframe per febbraio

In [ ]:
# Lista delle cartelle (sostituisci con i tuoi percorsi)
folders = [
    '/Users/boe/Desktop/Unimi/Tesi/code/SensorsData/subject_1/environmentals/Dishes_Glasses/',
    '/Users/boe/Desktop/Unimi/Tesi/code/SensorsData/subject_1/environmentals/FoodStorage',
    '/Users/boe/Desktop/Unimi/Tesi/code/SensorsData/subject_1/environmentals/Freezer',
    '/Users/boe/Desktop/Unimi/Tesi/code/SensorsData/subject_1/environmentals/HouseEntrance',
    '/Users/boe/Desktop/Unimi/Tesi/code/SensorsData/subject_1/environmentals/Microwave',
    '/Users/boe/Desktop/Unimi/Tesi/code/SensorsData/subject_1/environmentals/MotionBathroom',
    '/Users/boe/Desktop/Unimi/Tesi/code/SensorsData/subject_1/environmentals/MotionBedroom',
    '/Users/boe/Desktop/Unimi/Tesi/code/SensorsData/subject_1/environmentals/MotionKitchen',
    '/Users/boe/Desktop/Unimi/Tesi/code/SensorsData/subject_1/environmentals/MotionLivingroom',
    '/Users/boe/Desktop/Unimi/Tesi/code/SensorsData/subject_1/environmentals/PlugTvHall',
    '/Users/boe/Desktop/Unimi/Tesi/code/SensorsData/subject_1/environmentals/Refrigerator',
    '/Users/boe/Desktop/Unimi/Tesi/code/SensorsData/subject_1/environmentals/Shower_Hum_Temp_humidity',
    '/Users/boe/Desktop/Unimi/Tesi/code/SensorsData/subject_1/environmentals/Shower_Hum_Temp_temp',
    '/Users/boe/Desktop/Unimi/Tesi/code/SensorsData/subject_1/environmentals/Silverware',
    '/Users/boe/Desktop/Unimi/Tesi/code/SensorsData/subject_1/environmentals/Stove_Hum_Temp_humidity',
    '/Users/boe/Desktop/Unimi/Tesi/code/SensorsData/subject_1/environmentals/Stove_Hum_Temp_humidity',
    '/Users/boe/Desktop/Unimi/Tesi/code/SensorsData/subject_1/environmentals/WashingMachine'
]

# Nome del file che cerchiamo in ogni cartella
filename = "February_2024.csv"

# Lista per contenere tutti i DataFrame
dfs = []

# Legge il CSV da ogni cartella
for folder in folders:
    file_path = os.path.join(folder, filename)
    # Verifica se il file esiste prima di leggerlo
    if os.path.exists(file_path):
        try:
            df = pd.read_csv(file_path)
            dfs.append(df)
        except Exception as e:
            print(f"Errore nella lettura del file {file_path}: {str(e)}")
    else:
        print(f"File non trovato in: {folder}")

# Se abbiamo trovato almeno un file
if dfs:
    # Unisce tutti i DataFrame in uno solo
    df_finale_february = pd.concat(dfs, ignore_index=True)
    
    # Ordina il DataFrame per local_time
    df_finale_february = df_finale_february.sort_values('local_time')
    
    # Resetta l'indice dopo l'ordinamento
    df_finale_february = df_finale_february.reset_index(drop=True)
    
    print(f"Elaborati {len(dfs)} file CSV con successo")
else:
    print("Nessun file CSV trovato nelle cartelle specificate")

## Nottata del 2024-02-16 - 2024-05-17 // Withings: 11:50 - Granular : 08:34

In [ ]:
df_finale_february['local_time'] = pd.to_datetime(df_finale_february['local_time'])
start_time = '2024-02-17 00:00:00'
end_time = '2024-02-17 20:00:00'
WithigsTime = '2024-02-17 11:50:00'  # Primo orario per la linea verticale
Granulartime = '2024-02-17 08:34:00'  # Secondo orario per la linea verticale
plot_sensor_status(df_finale_february, start_time, end_time, WithigsTime, Granulartime)

### non avendo a disposizione alcun dato certo per stimare quale valore sia quello più preciso si decide di prendere il valore che ha lo startValue pù preciso, quindi quello dei dati aggregati Withings

# Controllo cosa è successo in data 2024-06-20 - 2024-06-21 perchè i dati aggregati dicono che ha dormito solo con l'orologio ma ho i dati anche nei granulari

## Caricamento del dataframe per giugno

In [ ]:
# Lista delle cartelle (sostituisci con i tuoi percorsi)
folders = [
    '/Users/boe/Desktop/Unimi/Tesi/code/SensorsData/subject_1/environmentals/Dishes_Glasses/',
    '/Users/boe/Desktop/Unimi/Tesi/code/SensorsData/subject_1/environmentals/FoodStorage',
    '/Users/boe/Desktop/Unimi/Tesi/code/SensorsData/subject_1/environmentals/Freezer',
    '/Users/boe/Desktop/Unimi/Tesi/code/SensorsData/subject_1/environmentals/HouseEntrance',
    '/Users/boe/Desktop/Unimi/Tesi/code/SensorsData/subject_1/environmentals/Microwave',
    '/Users/boe/Desktop/Unimi/Tesi/code/SensorsData/subject_1/environmentals/MotionBathroom',
    '/Users/boe/Desktop/Unimi/Tesi/code/SensorsData/subject_1/environmentals/MotionBedroom',
    '/Users/boe/Desktop/Unimi/Tesi/code/SensorsData/subject_1/environmentals/MotionKitchen',
    '/Users/boe/Desktop/Unimi/Tesi/code/SensorsData/subject_1/environmentals/MotionLivingroom',
    '/Users/boe/Desktop/Unimi/Tesi/code/SensorsData/subject_1/environmentals/PlugTvHall',
    '/Users/boe/Desktop/Unimi/Tesi/code/SensorsData/subject_1/environmentals/Refrigerator',
    '/Users/boe/Desktop/Unimi/Tesi/code/SensorsData/subject_1/environmentals/Shower_Hum_Temp_humidity',
    '/Users/boe/Desktop/Unimi/Tesi/code/SensorsData/subject_1/environmentals/Shower_Hum_Temp_temp',
    '/Users/boe/Desktop/Unimi/Tesi/code/SensorsData/subject_1/environmentals/Silverware',
    '/Users/boe/Desktop/Unimi/Tesi/code/SensorsData/subject_1/environmentals/Stove_Hum_Temp_humidity',
    '/Users/boe/Desktop/Unimi/Tesi/code/SensorsData/subject_1/environmentals/Stove_Hum_Temp_humidity',
    '/Users/boe/Desktop/Unimi/Tesi/code/SensorsData/subject_1/environmentals/WashingMachine'
]

# Nome del file che cerchiamo in ogni cartella
filename = "June_2024.csv"

# Lista per contenere tutti i DataFrame
dfs = []

# Legge il CSV da ogni cartella
for folder in folders:
    file_path = os.path.join(folder, filename)
    # Verifica se il file esiste prima di leggerlo
    if os.path.exists(file_path):
        try:
            df = pd.read_csv(file_path)
            dfs.append(df)
        except Exception as e:
            print(f"Errore nella lettura del file {file_path}: {str(e)}")
    else:
        print(f"File non trovato in: {folder}")

# Se abbiamo trovato almeno un file
if dfs:
    # Unisce tutti i DataFrame in uno solo
    df_finale_june = pd.concat(dfs, ignore_index=True)
    
    # Ordina il DataFrame per local_time
    df_finale_june = df_finale_june.sort_values('local_time')
    
    # Resetta l'indice dopo l'ordinamento
    df_finale_june = df_finale_june.reset_index(drop=True)
    
    print(f"Elaborati {len(dfs)} file CSV con successo")
else:
    print("Nessun file CSV trovato nelle cartelle specificate")

## Nottata del 2024-06-20 - 2024-06-21 // Withings: 23:59 - Granular : 23:59

In [ ]:
df_finale_june['local_time'] = pd.to_datetime(df_finale_june['local_time'])
start_time = '2024-06-10 12:00:00'
end_time = '2024-06-21 12:00:00'
WithigsTime = '2024-06-20 23:59:00'  # Primo orario per la linea verticale
Granulartime = '2024-06-20 23:59:00'  # Secondo orario per la linea verticale
plot_sensor_status(df_finale_june, start_time, end_time, WithigsTime, Granulartime)